Part 2 

In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import os
import time 
import urllib3, socket
from urllib3.connection import HTTPConnection


RESPONSE_SAVE_LOC = r'C:\Users\khali\OneDrive - Danmarks Tekniske Universitet\6. semester\Computational Social Science\Computational-Social-Science-Exercises\week2'
ids_list = list(np.load(r'C:\Users\khali\OneDrive - Danmarks Tekniske Universitet\6. semester\Computational Social Science\Computational-Social-Science-Exercises\week2\remaining_ids.npy',allow_pickle=True))

#Uses checkpoints in case connection to server is unstable.
if os.path.exists(fr'{RESPONSE_SAVE_LOC}/response.json'):
    with open(fr'{RESPONSE_SAVE_LOC}/response.json', "r") as f:
        # Load the contents of the file into a Python list
        response = json.load(f)
    response_items = []
    for item in response:
        if isinstance(item,dict):
            response_items.append(item)
    logged_ids = [ x['authorId'] for x in response_items]
else: response, logged_ids = [], []

#Converts all ids to int and removes duplicate requests
ids_list = [str(x) for x in ids_list]
logged_ids = [str(x) for x in logged_ids]
remaining_ids = list(set(ids_list) - set(logged_ids))

#use first half of remaining_ids:
remaining_ids = remaining_ids[0:int(len(remaining_ids)/2)]

#np.save(fr'{RESPONSE_SAVE_LOC}/remaining_ids.npy', remaining_ids)


#Get a list corresponding to second half of remaining_ids using length of remaining_ids:


#del ids_list, logged_ids, response_items

BASE_URL = 'https://api.semanticscholar.org/graph/'
VERSION = 'v1/'
RESOURCE = 'author/batch'
CHUNK_SIZE = 100

COMPLETE_URL = BASE_URL + VERSION + RESOURCE

def get_request(url,json=None, params=None):
    #Function to recursisevely call the API until we get a proper response.
    #time.sleep(2)
    responsex = requests.post(url, json=json, params=params, timeout=300)
    if responsex.status_code != 200:
        print(f'Error: {responsex.status_code}')
        print(f'Error: {responsex.json()}')
        #wait 5 seconds and try again
        #time.sleep(5)
        return get_request(url,json=json, params=params)
    else:
        return responsex


#When we have to send data ind, we use requests.post() and max author request size is 100 at a time.
params = {"fields": [
    "name,aliases,papers,papers.title,papers.abstract,papers.year,papers.externalIds,papers.s2FieldsOfStudy,papers.citationCount"]
}
print(f'Retrieving data, num requests:{len(remaining_ids)}')
for i in range(0, int(len(remaining_ids)), CHUNK_SIZE): # len(ids_list)
    idsx = remaining_ids[i:i + CHUNK_SIZE]
    idsx_json = {"ids" : idsx}
    responsex = get_request(COMPLETE_URL, json=idsx_json, params=params).json()

    if response == []:
        response = responsex
    else: 
        response += responsex
    print(f'{i+CHUNK_SIZE} requests')

    if i % (2*CHUNK_SIZE) == 0:
        json_string = json.dumps(response)

        with open(fr'{RESPONSE_SAVE_LOC}/response.json', "w") as f:
            f.write(json_string)

print('Data retrieved')

print(fr'RESPONSE saved to path:{RESPONSE_SAVE_LOC}/response.json')



After saving response, use it to make dataframes.

In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import os

RESPONSE_SAVE_LOC = '/work3/s204161'

#ids_list = list(np.load(r'/zhome/a7/0/155527/Desktop/s204161/Computational-Social-Science-Exercises/week2/author_ids.npy',allow_pickle=True))

with open(fr'{RESPONSE_SAVE_LOC}/response.json', "r") as f:
    # Load the contents of the file into a Python list
    response = json.load(f)


#-------------author_dataset----------------
author_dataset_dict_list = [] #[None for x in range(len(response))]
for counter, author in enumerate(response):
    paper_category_list = []
    citationCount = 0 #num times author was cited

    for paper in author.get('papers'):
        None if (paper['s2FieldsOfStudy'] == []) else (paper_category_list.append(paper['s2FieldsOfStudy'][0]['category']))
        citationCount += paper['citationCount']

    author_dataset_dict = {
        'authorId': author['authorId'],
        'name': author['name'],
        'aliases': author['aliases'],
        'citationCount': citationCount,
        'field': None if paper_category_list == [] else max(set(paper_category_list), key=paper_category_list.count)

    }
    author_dataset_dict_list.append(author_dataset_dict)

author_df = pd.DataFrame(author_dataset_dict_list)
#print(author_df)


#-----------------------paper dataset----------------

paper_dataset_dict_list = [] #[None for x in range(len(response))]
paper_id_dict = {}
for counter, author in enumerate(response):

    for paper in author.get('papers'):
        if paper['paperId'] not in paper_id_dict: # Only add paper if paper not already denoted using  other author.
            paper_id_dict[paper['paperId']] = len(paper_id_dict) #  Keep track by denoting the index of each paper in the data, chronologically.

            paper_dataset_dict = {
                    'paperId': paper['paperId'],
                    'title': paper['title'],
                    'year': paper['year'],
                    'externalId.DOI': paper['externalIds'],
                    'citationCount': paper['citationCount'], #num times 
                    'authorIds': [author['authorId']] ,
                    's2FieldsOfStudy': None if (paper['s2FieldsOfStudy'] == []) else (paper['s2FieldsOfStudy'][0]['category'])
                }
            paper_dataset_dict_list.append(paper_dataset_dict)
        else:
            current_paper_index = paper_id_dict[paper['paperId']]
            paper_dataset_dict_list[current_paper_index]['authorIds'].append(author['authorId'])

paper_df = pd.DataFrame(paper_dataset_dict_list)


## Paper abstract dataset.
paper_abstract_dataset_dict_list = [] 
paper_abstract_id_dict = {}
for counter, author in enumerate(response):

    #     ##this block might be superfluos if response not faulty.
    # if isinstance(author,str) == True:
    #     continue
    # elif author == None:
    #     #print(f'Entry {counter} is None.')
    #     continue
    
    for paper in author.get('papers'):
        if paper['paperId'] not in paper_abstract_id_dict:
            paper_abstract_id_dict[paper['paperId']] = len(paper_abstract_id_dict) #  Denote the index of each paper in the data, chronologically.

            paper_abstract_dataset_dict = {
                    'paperId': paper['paperId'],
                    'abstract': paper['abstract'],
                }
            paper_abstract_dataset_dict_list.append(paper_abstract_dataset_dict)


paper_abstract_df = pd.DataFrame(paper_abstract_dataset_dict_list)

#print(paper_df)

author_df.to_csv(r'/work3/s204161/author_df.csv')
paper_df.to_csv(r'/work3/s204161/paper_df.csv')
paper_abstract_df.to_csv(r'/work3/s204161/paper_astract_df.csv')

